In [ ]:
from gensim.models import KeyedVectors
from scipy.stats import spearmanr

# Load pretrained embeddings - GloVe
model = KeyedVectors.load_word2vec_format("glove.6B.100d.txt", binary=False, no_header=True)

def evaluate_word_similarity(word_pairs, human_scores):
    predicted_scores = []
    for w1, w2 in word_pairs:
        if w1 in model.key_to_index and w2 in model.key_to_index:
            similarity = model.similarity(w1, w2)
        else:
            similarity = 0  # Handling OOV words
        predicted_scores.append(similarity)

    spearman_corr, _ = spearmanr(human_scores, predicted_scores)
    return spearman_corr

# Example usage
word_pairs = []
human_scores = []
with open('dataset1.csv', 'r') as f:
    next(f)  # Skip header row
    for line in f:
        parts = line.strip().split(',')
        word_pairs.append((parts[0], parts[1]))
        human_scores.append(float(parts[3]))

print("Spearman Correlation:", evaluate_word_similarity(word_pairs, human_scores))


In [ ]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_contextual_embedding(word, sentence):
    inputs = tokenizer(sentence, return_tensors='pt')
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    word_index = sentence.split().index(word)
    return last_hidden_states[0][word_index].detach().numpy()

# Example usage
sentence = "The bank is on the river"
word = "bank"
embedding = get_contextual_embedding(word, sentence)
print("Contextual Embedding:", embedding)